<a href="https://colab.research.google.com/github/StefanATudose/NLP-Hackathon-2023/blob/main/Hackaton%20v2%20(0.30%20acc).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [8]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
os.chdir("/content/drive/My Drive/NitroNLP/Hackaton")
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer(language='romanian') # This one has support for Romanian too!

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocesare

In [11]:
seed = 19


import numpy as np
np.random.seed(seed)
np.random.RandomState(seed)

import random
random.seed(seed)

import torch
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

import tensorflow as tf
tf.random.set_seed(seed)

import jax
jax.random.PRNGKey(seed)

Array([ 0, 19], dtype=uint32)

In [12]:
df = pd.read_csv("./train_data.csv")
test_df = pd.read_csv("./test_data.csv")

In [13]:
def normalize_tokens(tokens):

    tokFinal = []
    for token in tokens:
        if token in string.punctuation:
            continue
        token = re.sub('ă', 'a', token)
        token = re.sub('â', 'a', token)
        token = re.sub('î', 'i', token)
        token = re.sub('ș', 's', token)
        token = re.sub('ț', 't', token)
        token = re.sub('\.', '', token)
        if token in stopwords.words("romanian"):
            continue
        if token == "":
            continue
        token = stemmer.stem(token)
        tokFinal.append(token)
    # tokFinal = [t for t in tokFinal if t not in string.punctuation]
    # tokFinal = [t for t in tokFinal if t not in stopwords.words("romanian")]
    # tokFinal = [stemmer.stem(t) for t in tokFinal]
    return tokFinal

In [ ]:
" ".join(normalize_tokens(tweet_tokenizer.tokenize(df["Text"][0])))

'înca trecut 28 😃 😃 feric 👍 🌺'

In [14]:
for line in range(len(df["Text"])):
    df.loc[line, "Text"] = df.loc[line, "Text"].lower()
    df.loc[line, "Text"] = " ".join(normalize_tokens(tweet_tokenizer.tokenize(df.loc[line, "Text"])))

for line in range(len(test_df["Text"])):
    test_df.loc[line, "Text"] = test_df.loc[line, "Text"].lower()
    test_df.loc[line, "Text"] = " ".join(normalize_tokens(tweet_tokenizer.tokenize(test_df.loc[line, "Text"])))

Din fericire în extaz!Ai dus covoareleeee?? Pune mîna și aspiră tavanul și pereții că doar n-o să stea aspiratorul ăla degeaba. Nu ne batem joc de bani, da!!!Vaiiii, ne-au furat "Răpirea din serai"! Bine dreaq că nu avem rembranzi și monezi! Dispar pînă nu o păgubesc de una.pwp!
['din', 'feric', 'extaz', 'ai', 'dus', 'covoarelee', 'pun', 'min', 'aspir', 'tavan', 'peret', 'n-o', 'ste', 'aspir', 'degeab', 'nu', 'bat', 'joc', 'ban', 'vai', 'ne-au', 'furat', 'rapir', 'sera', 'bin', 'dreaq', 'rembranz', 'monez', 'dispar', 'pagub', 'unapwp']


# Antrenare

In [15]:
train_df = df
# val_df = df[19000:20000]

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train_df["Text"])

train_data = vectorizer.transform(train_df["Text"])
# val_data = vectorizer.transform(val_df["Text"])
test_data = vectorizer.transform(test_df["Text"])

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
scaler.fit(train_data)

scaled_x_train = scaler.transform(train_data)
# scaled_x_val = scaler.transform(val_data)
scaled_x_test = scaler.transform(test_data)

In [17]:
from sklearn import svm

def svm_classifier(train_data, train_labels, test_data, c):
    linear_svm_classifier = svm.SVC(C=c, kernel='sigmoid') # sigmoid
    linear_svm_classifier.fit(train_data, train_labels)
    predicted_labels_train = linear_svm_classifier.predict(train_data)
    predicted_labels_test = linear_svm_classifier.predict(test_data)

    return predicted_labels_train, predicted_labels_test

def compute_accuracy(gt_labels, predicted_labels):
    accuracy = np.sum(predicted_labels == gt_labels) / len(predicted_labels)
    return accuracy


In [24]:
predicted_labels_train, predicted_labels_test = svm_classifier(scaled_x_train, train_df["Final Labels"], \
                                                               scaled_x_test, \
                                                               500)

In [ ]:
# print("Accuracy: ", compute_accuracy(train_df["Final Labels"], predicted_labels_train))
# print("Accuracy: ", compute_accuracy(val_df["Final Labels"], predicted_labels_test))

In [25]:
lista1 = list(range(len(predicted_labels_test)))

d = {'Id': lista1, 'Label': predicted_labels_test}
sub = pd.DataFrame(data=d)
sub.to_csv("./C75.csv", index=False)

sub.groupby('Label').agg(['count'])


,Id
,count
Label,
descriptive,598
direct,275
non-offensive,2029
offensive,217
reporting,11
